In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from pathlib import Path
import os
import datetime

In [2]:
%load_ext tensorboard

In [3]:
def onehot(y: np.ndarray, n_classes) -> np.ndarray:
        """Encode labels into one-hot representation

        Parameters
        ------------
        y : Target values. shape = [n_samples]

        Returns
        -----------
        onehot : array, shape = (n_samples, n_labels)
        """
        onehot : np.ndarray = np.zeros((n_classes, y.shape[0]))
        for idx, val in enumerate(y.astype(int)):
            onehot[val, idx] = 1.
        return onehot.T

In [14]:
### System ###
checkpoint_str = "test/cp_ep40_bsize100_adam.ckpt"
checkpoint_path = Path(checkpoint_str)
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Make sure not to overwrite existing models
try:
    if len(os.listdir(checkpoint_path)) > 0:
        checkpoint_str += '_'
except FileNotFoundError:
    pass

### CNN ###
CLASSES_CNN = 10
CHANNELS_CNN = 1
BATCH_SIZE_CNN = 100
EPOCHS_CNN = 40

In [15]:
# Load and shape the dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train : np.ndarray = x_train.reshape(x_train.shape + (CHANNELS_CNN,))
x_test : np.ndarray = x_test.reshape(x_test.shape + (CHANNELS_CNN,))
y_train : np.ndarray = onehot(y_train, CLASSES_CNN)
y_test : np.ndarray = onehot(y_test, CLASSES_CNN)

# The first parameter of 'shape' is the number of samples.
num_train_samples = x_train.shape[0]
num_batches = int(num_train_samples / BATCH_SIZE_CNN)

In [16]:
# Normalize data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [17]:
# Save the model every epoch, only if the model has improved
cp_callback = ModelCheckpoint(checkpoint_str,
                              monitor='accuracy',
                              save_best_only=True,
                              mode='max',
                              save_weights_only=False,
                              save_freq='epoch',
                              #save_freq=5*num_batches,
                              verbose=2)
tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir,
                                          histogram_freq=1)
callbacks = [cp_callback, tb_callback]

In [8]:
# Create a new model
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(3, 3),
          activation='relu', input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=(3, 3),
          activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=(3, 3),
          activation='relu'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(32, activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(CLASSES_CNN, activation='softmax'))

In [18]:
# Compile the currently loaded model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(0.0009),
              metrics=['accuracy'])

In [10]:
import subprocess
# Clear the logs folder before running the fit again
subprocess.run('removelogs.bat')

CompletedProcess(args='removelogs.bat', returncode=0)

In [19]:
# Train the currently loaded model
history = model.fit(x_train, y_train,
                    batch_size=BATCH_SIZE_CNN,
                    epochs=EPOCHS_CNN,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks)

Epoch 1/20
600/600 [==============================] - 43s 71ms/step - loss: 0.0078 - accuracy: 0.9977 - val_loss: 0.0268 - val_accuracy: 0.9946

Epoch 00001: accuracy improved from -inf to 0.99767, saving model to training5\cp_ep60_bsize100_adam.ckpt
INFO:tensorflow:Assets written to: training5\cp_ep60_bsize100_adam.ckpt\assets
Epoch 2/20
600/600 [==============================] - 40s 67ms/step - loss: 0.0068 - accuracy: 0.9982 - val_loss: 0.0321 - val_accuracy: 0.9935

Epoch 00002: accuracy improved from 0.99767 to 0.99817, saving model to training5\cp_ep60_bsize100_adam.ckpt
INFO:tensorflow:Assets written to: training5\cp_ep60_bsize100_adam.ckpt\assets
Epoch 3/20
600/600 [==============================] - 39s 66ms/step - loss: 0.0060 - accuracy: 0.9983 - val_loss: 0.0280 - val_accuracy: 0.9945

Epoch 00003: accuracy improved from 0.99817 to 0.99830, saving model to training5\cp_ep60_bsize100_adam.ckpt
INFO:tensorflow:Assets written to: training5\cp_ep60_bsize100_adam.ckpt\assets
Epoc

In [12]:
%tensorboard --logdir logs/fit/

Reusing TensorBoard on port 6006 (pid 13004), started 0:00:01 ago. (Use '!kill 13004' to kill it.)

In [26]:
# Evaluate the model on the training set
score = model.evaluate(x_train, y_train, verbose=0)
print('loss: ', score[0])
print('score: ', score[1])

loss:  0.002153349108994007
score:  0.9995333552360535


In [28]:
# Evaluate the model on the test set
score = model.evaluate(x_test, y_test, verbose=0)
print('loss: ', score[0])
print('score: ', score[1])

loss:  0.03178893402218819
score:  0.9929999709129333


In [22]:
# Log the data obtained by the last evaluation
with open('log.txt', 'a') as file:
    file.write('File name: ' + checkpoint_str + '\n')
    file.write('loss: ' + str(score[0]) + '\n')
    file.write('score: ' + str(score[1]) + '\n')
    file.write('accuracy: ')
    file.write(str(history.history['accuracy'][-1]) + '\n')
    file.write('\n')

    file.close()

In [25]:
# Load the model and print a summary of its structure
model = tf.keras.models.load_model('./training5/cp_ep40_bsize100_adam.ckpt')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        18464     
_________________________________________________________________
batch_normalization_1 (Batch (None, 11, 11, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 32)          9